# Data collection activities
All the data collection activities are automated using user defined functions retrievable in the folder `scripts`.

In [1]:
from bs4 import BeautifulSoup
import requests as requests
from scripts import user, item, discussion, link, poll, bounty, comment
import csv
from tqdm import tqdm 

# Item scraping
The following code saves items data into a csv file, provided a range of item codes fixed by the operator.

First of all we need to initialize all the files for data collection

## Initialization of csv files

In [2]:
# Post items
file_path_post = "../data/post.csv"
row_head_post = ["Title",
                 "Category",
                 "Item code",
                 "Banner data",
                 "Main link",
                 "Body links",
                 "Sats received by comments",
                 "Comments item code",
                 ]
    
with open(file_path_post, 'w', encoding='utf_8_sig', newline="") as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(row_head_post)

In [3]:
# Comment items
file_path_comment = "../data/comment.csv"
row_head_comment = ["Item code",
                    "Banner data",
                    "Comments item code",
                    ]
    
with open(file_path_comment, 'w', encoding='utf_8_sig', newline="") as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(row_head_comment)

In [4]:
# Try to scrape 150 different items rather than in 'progressive item mode'
from random import sample

sampled_items = sample([*range(1,200000)], 10)


In [5]:
print(sampled_items)

[151041, 25864, 166960, 106144, 32060, 64643, 62542, 63703, 31873, 70302]


## First attempt

In [ ]:
# for i in tqdm(sampled_items):
#     try:
#         # Provided a string returns a bs4.BeautifulSoup object
#         url_posts = f'https://stacker.news/items/{i}'
#         response = requests.get(url_posts)
#         response.raise_for_status()
#         soup = BeautifulSoup(response.text, 'html.parser')
#         
#         if item.detect_item_type(i, soup)=='comment':
#             entry = [None,
#                      str(i),
#                      comment.extract_banner(soup),
#                      comment.extract_body_links(soup),
#                      comment.extract_comment_stacked(soup),
#                      comment.extract_comment_item_code(soup)
#                      ]
#             
#             # Appends every new profile to a csv file in the provided path
#             try:
#                 with open(file_path_comment, 'a', encoding='utf_8_sig', newline="") as csvfile:
#                     csvwriter = csv.writer(csvfile)
#                     csvwriter.writerow(entry)
#             except:
#                 print('Error while processing data')
#         
#         elif item.detect_item_type(i, soup)=='discussion':
#             entry = [discussion.extract_title(soup),
#                      str(i),
#                      discussion.extract_banner(soup),
#                      discussion.extract_body_links(soup),
#                      discussion.extract_comment_stacked(soup),
#                      discussion.extract_comment_item_code(soup)
#                      ]
#             
#             # Appends every new profile to a csv file in the provided path
#             try:
#                 with open(file_path_discussion, 'a', encoding='utf_8_sig', newline="") as csvfile:
#                     csvwriter = csv.writer(csvfile)
#                     csvwriter.writerow(entry)
#             except:
#                 print('Error while processing data')
#         
#         elif item.detect_item_type(i, soup)=='link':
#             entry = [link.extract_title(soup),
#                      str(i),
#                      link.extract_banner(soup),
#                      link.extract_link(soup),
#                      link.extract_body_links(soup),
#                      link.extract_comment_stacked(soup),
#                      link.extract_comment_item_code(soup)
#                      ]
#             # Appends every new profile to a csv file in the provided path
#             try:
#                 with open(file_path_link, 'a', encoding='utf_8_sig', newline="") as csvfile:
#                     csvwriter = csv.writer(csvfile)
#                     csvwriter.writerow(entry)
#             except:
#                 print('Error while processing data')
#                 
#         elif item.detect_item_type(i, soup)=='poll':
#             entry = [poll.extract_title(soup),
#                      str(i),
#                      poll.extract_banner(soup),
#                      poll.extract_body_links(soup),
#                      poll.extract_comment_stacked(soup),
#                      poll.extract_comment_item_code(soup)
#                      ]
#             # Appends every new profile to a csv file in the provided path
#             try:
#                 with open(file_path_poll, 'a', encoding='utf_8_sig', newline="") as csvfile:
#                     csvwriter = csv.writer(csvfile)
#                     csvwriter.writerow(entry)
#             except:
#                 print('Error while processing data')
#         
#         elif item.detect_item_type(i, soup)=='bounty':
#             entry = [bounty.extract_title(soup),
#                      str(i),
#                      bounty.extract_banner(soup),
#                      bounty.extract_body_links(soup),
#                      bounty.extract_comment_stacked(soup),
#                      bounty.extract_comment_item_code(soup)
#                      ]
#             # Appends every new profile to a csv file in the provided path
#             try:
#                 with open(file_path_bounty, 'a', encoding='utf_8_sig', newline="") as csvfile:
#                     csvwriter = csv.writer(csvfile)
#                     csvwriter.writerow(entry)
#             except:
#                 print('Error while processing data')
#         
#     except:
#         continue


## Fixing the resulting csv and the data structure

### Columns to be used for all the post item scraping

- Title
- Category
- Item code
- Banner data
- Main link
- Body links
- Sats received by comments
- Comment item code

### Columns to be used for the comment item scraping
- Item code
- Banner data
- Comment item code

**NB**-> the `comment item code` in Comment item table could even be deleted, we can just keep it in order to eventually see the relationship between the comment and the comments to the specific comment

In [ ]:
# for i in tqdm(sampled_items):
#     try:
#         # Provided a string returns a bs4.BeautifulSoup object
#         url_posts = f'https://stacker.news/items/{i}'
#         response = requests.get(url_posts)
#         response.raise_for_status()
#         soup = BeautifulSoup(response.text, 'html.parser')
#         
#         if item.detect_item_type(i, soup)=='comment':
#             entry = [str(i),
#                      comment.extract_banner(soup),
#                      comment.extract_comment_item_code(soup)
#                      ]
#             
#             # Appends every new profile to a csv file in the provided path
#             try:
#                 with open(file_path_comment, 'a', encoding='utf_8_sig', newline="") as csvfile:
#                     csvwriter = csv.writer(csvfile)
#                     csvwriter.writerow(entry)
#             except:
#                 print('Error while processing data')
# 
#         elif item.detect_item_type(i, soup)=='link':
#             entry = [link.extract_title(soup),
#                      item.detect_item_type(i,soup),
#                      str(i),
#                      link.extract_banner(soup),
#                      link.extract_link(soup),
#                      link.extract_body_links(soup),
#                      link.extract_comment_stacked(soup),
#                      link.extract_comment_item_code(soup)
#                      ]
#             # Appends every new profile to a csv file in the provided path
#             try:
#                 with open(file_path_post, 'a', encoding='utf_8_sig', newline="") as csvfile:
#                     csvwriter = csv.writer(csvfile)
#                     csvwriter.writerow(entry)
#             except:
#                 print('Error while processing data')
#         
#         elif item.detect_item_type(i, soup)=='discussion':
#             entry = [discussion.extract_title(soup),
#                      item.detect_item_type(i,soup),
#                      str(i),
#                      discussion.extract_banner(soup),
#                      None,
#                      discussion.extract_body_links(soup),
#                      discussion.extract_comment_stacked(soup),
#                      discussion.extract_comment_item_code(soup)
#                      ]
#             
#             # Appends every new profile to a csv file in the provided path
#             try:
#                 with open(file_path_post, 'a', encoding='utf_8_sig', newline="") as csvfile:
#                     csvwriter = csv.writer(csvfile)
#                     csvwriter.writerow(entry)
#             except:
#                 print('Error while processing data')
#                         
#         elif item.detect_item_type(i, soup)=='poll':
#             entry = [poll.extract_title(soup),
#                      item.detect_item_type(i, soup),
#                      str(i),
#                      poll.extract_banner(soup),
#                      None,
#                      poll.extract_body_links(soup),
#                      poll.extract_comment_stacked(soup),
#                      poll.extract_comment_item_code(soup)
#                      ]
#             # Appends every new profile to a csv file in the provided path
#             try:
#                 with open(file_path_post, 'a', encoding='utf_8_sig', newline="") as csvfile:
#                     csvwriter = csv.writer(csvfile)
#                     csvwriter.writerow(entry)
#             except:
#                 print('Error while processing data')
#         
#         elif item.detect_item_type(i, soup)=='bounty':
#             entry = [bounty.extract_title(soup),
#                      item.detect_item_type(i, soup),
#                      str(i),
#                      bounty.extract_banner(soup),
#                      None,
#                      bounty.extract_body_links(soup),
#                      bounty.extract_comment_stacked(soup),
#                      bounty.extract_comment_item_code(soup)
#                      ]
#             # Appends every new profile to a csv file in the provided path
#             try:
#                 with open(file_path_post, 'a', encoding='utf_8_sig', newline="") as csvfile:
#                     csvwriter = csv.writer(csvfile)
#                     csvwriter.writerow(entry)
#             except:
#                 print('Error while processing data')
#         
#     except:
#         continue


### Optimizing for the similarities between item post types

In the following chunck the functions applied are clustered for similarities, leading to three blocks:
- Comment
- Link
- Discussion/poll/bounty


In [6]:
for i in tqdm(sampled_items):
    try:
        # Provided a string returns a bs4.BeautifulSoup object
        url_posts = f'https://stacker.news/items/{i}'
        response = requests.get(url_posts)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        if item.detect_item_type(i, soup)=='comment':
            entry = [str(i),
                     comment.extract_banner(soup),
                     comment.extract_comment_item_code(soup)
                     ]
            
            # Appends every new profile to a csv file in the provided path
            try:
                with open(file_path_comment, 'a', encoding='utf_8_sig', newline="") as csvfile:
                    csvwriter = csv.writer(csvfile)
                    csvwriter.writerow(entry)
            except:
                print('Error while processing data')

        elif item.detect_item_type(i, soup)=='link':
            entry = [link.extract_title(soup),
                     item.detect_item_type(i,soup),
                     str(i),
                     link.extract_banner(soup),
                     link.extract_link(soup),
                     link.extract_body_links(soup),
                     link.extract_comment_stacked(soup),
                     link.extract_comment_item_code(soup)
                     ]
            # Appends every new profile to a csv file in the provided path
            try:
                with open(file_path_post, 'a', encoding='utf_8_sig', newline="") as csvfile:
                    csvwriter = csv.writer(csvfile)
                    csvwriter.writerow(entry)
            except:
                print('Error while processing data')
        
        elif item.detect_item_type(i, soup) in ['discussion', 'poll', 'bounty']:
            entry = [discussion.extract_title(soup),
                     item.detect_item_type(i,soup),
                     str(i),
                     discussion.extract_banner(soup),
                     None,
                     discussion.extract_body_links(soup),
                     discussion.extract_comment_stacked(soup),
                     discussion.extract_comment_item_code(soup)
                     ]
            
            # Appends every new profile to a csv file in the provided path
            try:
                with open(file_path_post, 'a', encoding='utf_8_sig', newline="") as csvfile:
                    csvwriter = csv.writer(csvfile)
                    csvwriter.writerow(entry)
            except:
                print('Error while processing data')
        
    except:
        continue


100%|██████████| 10/10 [00:35<00:00,  3.52s/it]
